In [ ]:
import random
import torch
import pandas as pd
import os
import time
import datasets
from datasets import load_dataset
from IPython.display import display, HTML
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer,DataCollatorForLanguageModeling
from peft import PeftModel,LoraConfig,get_peft_model
#from peft import prepare_model_for_int8_training

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model_path = r'D:/MODELS/deepseek-ai/DeepSeek-R1-Distill-Qwen-1___5B'

#lora_path = './output/llama3_1_instruct_lora/checkpoint-699' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()


In [ ]:
model

In [ ]:
dataset = load_dataset('csv', data_files='C:/Users/lenovo/Desktop/AI project/Seprate_docx/4-6岁故事范文.csv')
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(dataset['train'])

In [ ]:
dataset

In [5]:
# 获取当前模型占用的 GPU显存（差值为预留给 PyTorch 的显存）
memory_footprint_bytes = model.get_memory_footprint()
memory_footprint_mib = memory_footprint_bytes / (1024 ** 3)  # 转换为 GB

print(f"{memory_footprint_mib:.2f}GB")

3.31GB


In [6]:
config=LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model=get_peft_model(model,config)

In [7]:
tokenized_dataset = dataset.map(lambda samples: tokenizer(samples["instruction"]), batched=True)

Map: 100%|██████████| 399/399 [00:00<00:00, 19024.71 examples/s]


In [8]:
# 数据收集器，用于处理语言模型的数据，这里设置为不使用掩码语言模型(MLM)
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
model_dir = "models"
current_time = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
model_path = f"model_{current_time}"  # 例如 'model_2025-02-27_15-30-49'

training_args = TrainingArguments(
        output_dir = 
        
        , # 指定模型输出和保存的目录
        per_device_train_batch_size=4,  # 每个设备上的训练批量大小
        learning_rate=2e-4,  # 学习率
        fp16=False,  # 启用混合精度训练，可以提高训练速度，同时减少内存使用
        logging_steps=20,  # 指定日志记录的步长，用于跟踪训练进度
        max_steps=100, # 最大训练步长
         num_train_epochs=1  # 训练的总轮数
    )

In [10]:
trainer = Trainer(
    model=model,  # 指定训练时使用的模型
    train_dataset=tokenized_dataset["train"],  # 指定训练数据集
    args=training_args,
    data_collator=data_collator,
)

In [11]:
trainer.train()

Step,Training Loss
20,3.665000
40,2.088500
60,0.637000
80,0.009400
100,0.001000


TrainOutput(global_step=100, training_loss=1.2802021817862987, metrics={'train_runtime': 12.4017, 'train_samples_per_second': 32.254, 'train_steps_per_second': 8.063, 'total_flos': 18496106818560.0, 'train_loss': 1.2802021817862987, 'epoch': 1.0})

In [ ]:
model_path = r"D:/MODELS/deepseek-ai"

#trainer.save_model(model_path)
model.save_pretrained(model_path)

In [12]:
lora_model = trainer.model

In [14]:
text = "写一篇童话故事（森林，小猪，团结），直接开始写"
inputs = tokenizer(text, return_tensors="pt").to(0)

out = lora_model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(out[0], skip_special_tokens=True))   

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


写一篇童话故事（森林，小猪，团结），直接开始写吧。
要求：1. 故事要有童话色彩，简单易记。
2. 语言要生动，有细节描写。
3. 内容要围绕“森林里的小猪，团结的小群体”展开。
4. 请您按要求写一篇童话故事，字数大约是300字左右。
好的，我来试着写写看。首先，故事的背景是森林里的小猪，团结的小群体。那么，小猪是什么呢？可能是一个比较内向的小猪，或者是一个比较有爱心的小猪。

然后，故事要有童话色彩，简单易记。所以，我需要把故事的情节写得简单，有节奏感。可能从一个温馨的场景开始，比如小猪在森林里玩耍，或者遇到什么有趣的事情。

接下来，语言要生动，有细节描写。所以，我要多写一些小猪的对话，森林里的景色，小猪的内心活动，以及整个故事的氛围。




In [ ]:
torch.cuda.empty_cache()